In [5]:
import os
import mlflow
from dotenv import load_dotenv
load_dotenv(override=True)

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000
YOUR_NAME = "Slava"
assert YOUR_NAME, "enter your name"

EXPERIMENT_NAME = f"test_connection_experiment_{YOUR_NAME}"
RUN_NAME = f"test_connection_run"

METRIC_NAME = "test_metric"
METRIC_VALUE = 0

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id

    mlflow.log_metric(METRIC_NAME, METRIC_VALUE)


In [6]:
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
run = mlflow.get_run(run_id)

assert "active" == experiment.lifecycle_stage
assert mlflow.get_run(run_id)
assert METRIC_VALUE == run.data.metrics[METRIC_NAME]